Modules and Packaging
====

At some point, you will want to organize and distribute your code library for the whole world to share, preferably on PyPI so that it is pip installable.

## References

This notebook shows a bare-bones version of creating and distributing a project to PyPI. Please follow the instructions in the official documentations. For convenience, you can use the sample project as a template. 

- [Packaging and Distributing Projects](https://packaging.python.org/tutorials/distributing-packages/)
- [A sample Python project](https://github.com/pypa/sampleproject)

For more about how to organize the structure of your package 

- [Official tutorial on packages](https://docs.python.org/3/tutorial/modules.html#packages)

If you are still confused about what `__init__.py` does, this [blog post](and the mysterious `__init__.py`, see) might help.

## Install packages we will use for packaging

In [ ]:
! pip install -U pip
! pip install twine

## Modules

In Pythoh, any `.py` file is a module in that it can be imported. Because the interpreter runs the entrie file when a moudle is imported, it is traditional to use a guard to ignore code that should only run when the file is executed as a script.

In [1]:
%%file foo.py
"""
When this file is imported with `import foo`,
only `useful_func1()` and `useful_func()` are loaded, 
and the test code `assert ...` is ignored. However,
when we run foo.py as a script `python foo.py`, then
the two assert statements are run.
Most commonly, the code under `if __naem__ == '__main__':`
consists of simple examples or test cases for the functions
defined in the moule.
"""

def useful_func1():
    pass

def useful_fucn2():
    pass

if __name__ == '__main__':
    assert(useful_func1() is None)
    assert(useful_fucn2() is None)

Overwriting foo.py


### Organization of files in a module

When the number of files you write grow large, you will probably want to orgnize them into their own directory structure. To make a folder a  module, you just need to include a file named `__init__.py` in the folder. This file can be empty. For example, here is a module named `pkg` with sub-modules `sub1` and `sub2`.

```
./pkg:
__init__.py	foo.py		sub1		sub2

./pkg/sub1:
__init__.py		more_sub1_stuff.py	sub1_stuff.py

./pkg/sub2:
__init__.py	sub2_stuff.py
```


In [2]:
import pkg.foo as foo

In [3]:
foo.f1()

1

In [4]:
import pkg

In [5]:
pkg.foo.f1()

1

#### How to import a module at the same level

Within a package, we need to use absolute path names for importing other modules in the same directory. This prevents confusion as to whether you want to import a system moudle with the same name. For example, `foo.sub1.more_sub1_stuff.py` imports functions from `foo.sub1.sub1_stuff.py`

In [6]:
! cat pkg/sub1/more_sub1_stuff.py

from pkg.sub1.sub1_stuff import g1, g2

def g3():
    return 'g3 uses %s, %s' % (g1(), g2())



In [7]:
from pkg.sub1.more_sub1_stuff import g3

g3()

'g3 uses g1, g2'

#### How to import a moudle at a different level

Again, just use absolute paths. For example, `sub2_stuff.py` in the `sub2` directory uses functions from `sub1_stuff.py` in the `sub1` directory:

In [8]:
! cat pkg/sub2/sub2_stuff.py

from pkg.sub1.sub1_stuff import g1, g2

def h1():
    return g1()

def h2():
    return g1() + g2()


In [9]:
from pkg.sub2.sub2_stuff import h2

h2()

'g1g2'

## Distributing your package

Suppose we want to distribute our code as a library (for example, on PyPI so that it cnn be installed with `pip`). Let's create an `sta663-<username>` (the username part is just to avoid name conflicts) library containing the `pkg` package and some other files:

- `README.md`: some information about the library
- `sta663.py`: a standalone module
- `run_sta663.py`: a script (intended for use as `python run_sta663.py`)

In [10]:
! ls -R sta663

sta663:
__init__.py  pkg	 run_sta663.py	sta663.egg-info  sub1
MANIFEST     README.txt  setup.py	sta663.py	 sub2

sta663/pkg:
foo.py	__init__.py  sub1  sub2

sta663/pkg/sub1:
__init__.py  more_sub1_stuff.py  sub1_stuff.py

sta663/pkg/sub2:
__init__.py  sub2_stuff.py

sta663/sta663.egg-info:
dependency_links.txt  PKG-INFO	SOURCES.txt  top_level.txt

sta663/sub1:
__intit__.py

sta663/sub2:
__iniit__.py


In [11]:
! cat sta663/run_sta663.py

import pkg.foo as foo
from pkg.sub1.more_sub1_stuff import g3
from pkg.sub2.sub2_stuff import h2

print foo.f1()
print g3()
print h2()


### Using distutils

All we need to do is to write a `setup.py` file.

In [12]:
%%file sta663/setup.py
from setuptools import setup
import setuptools
setup(name = "sta663-cliburn",
      version = "1.0",
      author='Cliburn Chan',
      author_email='cliburn.chan@duke.edu',
      url='http://people.duke.edu/~ccc14/sta-663-2018/',
      py_modules = ['sta663'],
      packages=setuptools.find_packages(),
      scripts = ['run_sta663.py'],
      python_requires='>=3',
      )

Overwriting sta663/setup.py


### Build a source archive for distribution

In [13]:
%%bash

cd sta663
python setup.py sdist
cd -

running sdist
running egg_info
creating sta663_cliburn.egg-info
writing sta663_cliburn.egg-info/PKG-INFO
writing dependency_links to sta663_cliburn.egg-info/dependency_links.txt
writing top-level names to sta663_cliburn.egg-info/top_level.txt
writing manifest file 'sta663_cliburn.egg-info/SOURCES.txt'
reading manifest file 'sta663_cliburn.egg-info/SOURCES.txt'
writing manifest file 'sta663_cliburn.egg-info/SOURCES.txt'
running check
creating sta663-cliburn-1.0
creating sta663-cliburn-1.0/pkg
creating sta663-cliburn-1.0/pkg/sub1
creating sta663-cliburn-1.0/pkg/sub2
creating sta663-cliburn-1.0/sta663_cliburn.egg-info
copying files to sta663-cliburn-1.0...
copying README.txt -> sta663-cliburn-1.0
copying run_sta663.py -> sta663-cliburn-1.0
copying setup.py -> sta663-cliburn-1.0
copying sta663.py -> sta663-cliburn-1.0
copying pkg/__init__.py -> sta663-cliburn-1.0/pkg
copying pkg/foo.py -> sta663-cliburn-1.0/pkg
copying pkg/sub1/__init__.py -> sta663-cliburn-1.0/pkg/sub1
copying pkg/sub1/mo

In [14]:
! ls -R sta663

sta663:
dist	     pkg	    setup.py		     sta663.py
__init__.py  README.txt     sta663_cliburn.egg-info  sub1
MANIFEST     run_sta663.py  sta663.egg-info	     sub2

sta663/dist:
sta663-cliburn-1.0.tar.gz

sta663/pkg:
foo.py	__init__.py  sub1  sub2

sta663/pkg/sub1:
__init__.py  more_sub1_stuff.py  sub1_stuff.py

sta663/pkg/sub2:
__init__.py  sub2_stuff.py

sta663/sta663_cliburn.egg-info:
dependency_links.txt  PKG-INFO	SOURCES.txt  top_level.txt

sta663/sta663.egg-info:
dependency_links.txt  PKG-INFO	SOURCES.txt  top_level.txt

sta663/sub1:
__intit__.py

sta663/sub2:
__iniit__.py


### Distribution

You can now distribute `sta663-1.0.tar.gz` to somebody else for installation in the usual way.

In [15]:
%%bash

cp sta663/dist/sta663_cliburn-1.0-py3.6.egg/tmp
cd /tmp
tar xzf sta663_cliburn-1.0-py3.6.egg 
cd sta663-1.0
python setup.py install

cp: cannot stat ‘sta663/dist/sta663_cliburn-1.0-py3.6.egg’: No such file or directory
tar (child): sta663_cliburn-1.0-py3.6.egg: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
bash: line 5: cd: sta663-1.0: No such file or directory
python: can't open file 'setup.py': [Errno 2] No such file or directory


### See `__init__.py` to understand what can be imported from where

In [16]:
! cat sta663/__init__.py

import pkg
from pkg.sub1.sub1_stuff import g1
 


In [17]:
! cat sta663/pkg/__init__.py

In [18]:
! cat sta663/pkg/sub1/__init__.py

from sta663.pkg.sub1.sub1_stuff import g1, g2
from sta663.pkg.sub1.more_sub1_stuff import g3


In [19]:
! cat sta663/pkg/sub2/__init__.py

from sta663.pkg.sub2.sub2_stuff import h1, h2


### Using imports from the `sta663` package

In [20]:
import sta663

In [21]:
[x for x in dir(sta663) if not x.startswith('__')]

['g1', 'pkg']

In [22]:
sta663.g1()

'g1'

In [23]:
from sta663 import pkg

In [24]:
[x for x in dir(pkg) if not x.startswith('__')]

['foo', 'sub1', 'sub2']

In [25]:
from sta663.pkg import sub1, sub2

In [26]:
[x for x in dir(sub1) if not x.startswith('__')]

['g1', 'g2', 'g3', 'more_sub1_stuff', 'sub1_stuff']

In [27]:
[x for x in dir(sub2) if not x.startswith('__')]

['h1', 'h2', 'sub2_stuff']

In [28]:
sub1.g1(), sub1.g2(), sub1.g3()

('g1', 'g2', 'g3 uses g1, g2')

In [29]:
sub2.h1(), sub2.h2()

('g1', 'g1g2')

In [30]:
sta663.pkg.sub2.sub2_stuff.h1()

'g1'

In [31]:
sta663.pkg.sub2.sub2_stuff.h2()

'g1g2'

#### Distributing to PyPI

For testing, please upload to TestPyPI which is cleaned on a regular basis. See instructions at 
https://packaging.python.org/guides/using-testpypi/#using-test-pypi

- **Note 1**: You need to confirm your email address after registration.
- **Note 2**: You can easily delete any uploaded packages by logging in at https://test.pypi.org.

When your package is ready for public release, you can upload to PyPI. See instructions at
https://packaging.python.org/tutorials/distributing-packages/#id78

In [32]:
%%bash

export TWINE_USERNAME='cliburn' 
export TWINE_PASSWORD=''
twine upload --repository-url https://test.pypi.org/legacy/ sta663/dist/*

Uploading distributions to https://test.pypi.org/legacy/
Uploading sta663-cliburn-1.0.tar.gz
100%|██████████| 3.85k/3.85k [00:00<00:00, 13.3kB/s]


HTTPError: 403 Client Error: Invalid or non-existent authentication information. for url: https://test.pypi.org/legacy/


In [33]:
%%bash

pip install --index-url https://test.pypi.org/simple/ sta663

Looking in indexes: https://test.pypi.org/simple/


  Could not find a version that satisfies the requirement sta663 (from versions: )
No matching distribution found for sta663


In [2]:
import fastica

ModuleNotFoundError: No module named 'fastica'